In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit 
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import top_k_accuracy_score

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Abro datasets y hago split en base a artistas al azar. Genero sets de entrenamiento, validación y testeo.

In [5]:
df_train = pd.read_parquet('/content/drive/MyDrive/ODD tp 1 dataset/train.parquet')

In [6]:
df_test = pd.read_parquet('/content/drive/MyDrive/ODD tp 1 dataset/test.parquet')

In [7]:
df_train.head()

,track_name,lyric,genre,language,popularity,artist,a_genres,a_songs,a_popularity,did,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Halo,Remember those walls I built\nWell baby they'r...,Dance,en,79,Beyoncé,Pop; R&B; Black Music,276.0,205.5,NaN,...,0.720,0.0000,B,0.0563,-5.908,Minor,0.0628,79.983,4/4,0.472
1,Halo,Remember those walls I built\nWell baby they'r...,Pop,en,80,Beyoncé,Pop; R&B; Black Music,276.0,205.5,NaN,...,0.720,0.0000,B,0.0563,-5.908,Minor,0.0628,79.983,4/4,0.472
2,Halo,Remember those walls I built\nWell baby they'r...,R&B,en,80,Beyoncé,Pop; R&B; Black Music,276.0,205.5,NaN,...,0.720,0.0000,B,0.0563,-5.908,Minor,0.0628,79.983,4/4,0.472
3,Partition,Driver roll up the partition please\nDriver ro...,Dance,en,71,Beyoncé,Pop; R&B; Black Music,276.0,205.5,NaN,...,0.441,0.0726,B,0.3060,-11.523,Minor,0.2910,185.571,4/4,0.174
4,Partition,Driver roll up the partition please\nDriver ro...,Pop,en,71,Beyoncé,Pop; R&B; Black Music,276.0,205.5,NaN,...,0.441,0.0726,B,0.3060,-11.523,Minor,0.2910,185.571,4/4,0.174


In [10]:
df_train.columns

Index(['track_name', 'lyric', 'genre', 'language', 'popularity', 'artist',
       'a_genres', 'a_songs', 'a_popularity', 'did', 's-label', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'valence'],
      dtype='object')

In [ ]:
splitter = GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 26)
split = splitter.split(df_train, groups=df_train['artist'])
train_inds, valid_inds = next(split)

train = df_train.iloc[train_inds]
valid = df_train.iloc[valid_inds]

In [ ]:
y_train = train.genre
X_train = train.drop(["track_name", "a_genres", "did", "genre", 'artist'], axis = 1)

In [ ]:
y_valid = valid.genre
X_valid = valid.drop(["track_name", "a_genres", "did", "genre", 'artist'], axis = 1)

In [ ]:
y_test = df_test.genre
X_test = df_test.drop(["track_name", "a_genres", "did", "genre", 'artist'], axis = 1)

# Imputación de nulos + OHE

In [ ]:
# Imputo nulos de Language y despues hago OHE

In [ ]:
X_train['language'] = X_train['language'].fillna(X_train['language'].mode()[0])
X_test['language'] = X_test['language'].fillna(X_train['language'].mode()[0])

In [ ]:
encoder = OneHotEncoder(drop='first', handle_unknown='ignore' )
lenguaje_train = encoder.fit_transform(X_train[['language']]).todense().astype(int)
encoder.get_feature_names_out()

array(['language_de', 'language_en', 'language_es', 'language_fr',
       'language_ga', 'language_gl', 'language_is', 'language_it',
       'language_pt', 'language_rw'], dtype=object)

In [ ]:
lenguaje_valid = encoder.transform(X_valid[['language']]).todense().astype(int)
encoder.get_feature_names_out()

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:174: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  UserWarning,


array(['language_de', 'language_en', 'language_es', 'language_fr',
       'language_ga', 'language_gl', 'language_is', 'language_it',
       'language_pt', 'language_rw'], dtype=object)

In [ ]:
lenguaje_test = encoder.transform(X_test[['language']]).todense().astype(int)
encoder.get_feature_names_out()

array(['language_de', 'language_en', 'language_es', 'language_fr',
       'language_ga', 'language_gl', 'language_is', 'language_it',
       'language_pt', 'language_rw'], dtype=object)

In [ ]:
X_train = X_train.reset_index().drop(['index', 'language'], axis=1).join(pd.DataFrame(lenguaje_train, columns = ('language_de', 'language_en', 'language_es', 'language_fr',
       'language_ga', 'language_gl', 'language_is', 'language_it',
       'language_pt', 'language_rw')))
X_valid = X_valid.reset_index().drop(['index', 'language'], axis=1).join(pd.DataFrame(lenguaje_valid, columns = ('language_de', 'language_en', 'language_es', 'language_fr',
       'language_ga', 'language_gl', 'language_is', 'language_it',
       'language_pt', 'language_rw')))
X_test = X_test.reset_index().drop(['index', 'language'], axis=1).join(pd.DataFrame(lenguaje_test, columns = ('language_de', 'language_en', 'language_es', 'language_fr',
       'language_ga', 'language_gl', 'language_is', 'language_it',
       'language_pt', 'language_rw')))

In [ ]:
# Lleno los nulos con la mediana de s-label de X_Train

In [ ]:
X_valid['s-label'] = X_valid['s-label'].fillna(X_train['s-label'].median())
X_train['s-label'] = X_train['s-label'].fillna(X_train['s-label'].median())
X_test['s-label'] = X_test['s-label'].fillna(X_train['s-label'].median())

In [ ]:
# Hago OHE a la categoria "key"

In [ ]:
key_train =  encoder.fit_transform(X_train[['key']]).todense().astype(int)
encoder.get_feature_names_out()

array(['key_A#', 'key_B', 'key_C', 'key_C#', 'key_D', 'key_D#', 'key_E',
       'key_F', 'key_F#', 'key_G', 'key_G#'], dtype=object)

In [ ]:
key_valid =  encoder.transform(X_valid[['key']]).todense().astype(int)
encoder.get_feature_names_out()

array(['key_A#', 'key_B', 'key_C', 'key_C#', 'key_D', 'key_D#', 'key_E',
       'key_F', 'key_F#', 'key_G', 'key_G#'], dtype=object)

In [ ]:
key_test =  encoder.transform(X_test[['key']]).todense().astype(int)
encoder.get_feature_names_out()

array(['key_A#', 'key_B', 'key_C', 'key_C#', 'key_D', 'key_D#', 'key_E',
       'key_F', 'key_F#', 'key_G', 'key_G#'], dtype=object)

In [ ]:
X_train = X_train.reset_index().drop(['index', 'key'], axis=1).join(pd.DataFrame(key_train, columns = ('key_A#', 'key_B', 'key_C', 'key_C#', 'key_D', 'key_D#', 'key_E',
       'key_F', 'key_F#', 'key_G', 'key_G#')))
X_valid = X_valid.reset_index().drop(['index', 'key'], axis=1).join(pd.DataFrame(key_valid, columns = ('key_A#', 'key_B', 'key_C', 'key_C#', 'key_D', 'key_D#', 'key_E',
       'key_F', 'key_F#', 'key_G', 'key_G#')))
X_test = X_test.reset_index().drop(['index', 'key'], axis=1).join(pd.DataFrame(key_test, columns = ('key_A#', 'key_B', 'key_C', 'key_C#', 'key_D', 'key_D#', 'key_E',
       'key_F', 'key_F#', 'key_G', 'key_G#')))

# Mean Encoding

In [ ]:
# Paso a 0 y 1 una columna categorica y luego aprovecho eso para hacer mean encoding de otra columna categorica

In [ ]:
X_train.loc[(X_train['mode'] == 'Major'),'mode']=int(1)
X_train.loc[(X_train['mode'] == 'Minor'),'mode']=int(0)

X_valid.loc[(X_valid['mode'] == 'Major'),'mode']=int(1)
X_valid.loc[(X_valid['mode'] == 'Minor'),'mode']=int(0)

X_test.loc[(X_test['mode'] == 'Major'),'mode']=int(1)
X_test.loc[(X_test['mode'] == 'Minor'),'mode']=int(0)

In [ ]:
X_train['mode'] = X_train['mode'].astype(int)
X_valid['mode'] = X_valid['mode'].astype(int)
X_test['mode'] = X_test['mode'].astype(int)

In [ ]:
mean_encoding = X_train.groupby(['time_signature'])['mode'].mean().to_dict()
  
X_train['time_signature'] =  X_train['time_signature'].map(mean_encoding)
X_valid['time_signature'] =  X_valid['time_signature'].map(mean_encoding)
X_test['time_signature'] =  X_test['time_signature'].map(mean_encoding)

# TF-IDF Vectorizer

In [ ]:
# Lleno los nulos con un espacio vacío y después vectorizo y joineo. 

In [ ]:
X_train["lyric"] = X_train["lyric"].fillna("")
X_valid["lyric"] = X_valid["lyric"].fillna("")
X_test["lyric"] = X_test["lyric"].fillna("")

In [ ]:
countIDF = TfidfVectorizer(lowercase=True, stop_words='english', max_features=50)

In [ ]:
vectorizado_train = countIDF.fit_transform(X_train["lyric"]).todense().astype(int)
vectorizado_valid = countIDF.transform(X_valid["lyric"]).todense().astype(int)
vectorizado_test = countIDF.transform(X_test["lyric"]).todense().astype(int)

In [ ]:
X_train = X_train.reset_index().drop(['index', 'lyric'], axis=1).join(pd.DataFrame(vectorizado_train))
X_valid = X_valid.reset_index().drop(['index', 'lyric'], axis=1).join(pd.DataFrame(vectorizado_valid))
X_test = X_test.reset_index().drop(['index', 'lyric'], axis=1).join(pd.DataFrame(vectorizado_test))

**Normalizo valores**

In [ ]:
escalador = MinMaxScaler()

X_train_escalado = escalador.fit_transform(X_train)
X_train = pd.DataFrame(X_train_escalado, columns = X_train.columns)

X_valid_escalado = escalador.transform(X_valid)
X_valid = pd.DataFrame(X_valid_escalado, columns = X_valid.columns)

X_test_escalado = escalador.transform(X_test)
X_test = pd.DataFrame(X_test_escalado, columns = X_test.columns)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


# Busco hiper parametros y hago modelo de Random Forest

In [ ]:
rf_model_base = RandomForestClassifier(random_state=26)
modelo_rf_base = rf_model_base.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
print(f'El mejor score de validación con parametros default obtenido es {top_k_accuracy_score(y_valid, rf_model_base.predict_proba(X_valid), k=2, labels=modelo_rf_base.classes_)}')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


El mejor score de validación con parametros default obtenido es 0.503609541745135


In [ ]:
# Busqueda de hiperparametros

model = RandomForestClassifier(random_state=26)
espacio_busqueda = { 'min_samples_split' : [2, 5, 10, 15],
                    'min_samples_leaf' : [1, 3, 6],
 'max_depth' : [3, 5, 10, 15, 20],
 'n_estimators' : [10, 20, 30, 50, 100],
 'max_features' : [10, 25, 50, 70],
 }

# Defino y ejecuto busqueda de hiperparametros

busqueda_parametros = RandomizedSearchCV(model, espacio_busqueda, n_iter=250, scoring='accuracy', n_jobs=1, cv=5, verbose=10, random_state=26)
resultados = busqueda_parametros.fit(X_train, y_train)

Fitting 5 folds for each of 250 candidates, totalling 1250 fits
[CV 1/5; 1/250] START max_depth=20, max_features=10, min_samples_leaf=1, min_samples_split=10, n_estimators=10


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


[CV 1/5; 1/250] END max_depth=20, max_features=10, min_samples_leaf=1, min_samples_split=10, n_estimators=10;, score=0.319 total time=   0.5s
[CV 2/5; 1/250] START max_depth=20, max_features=10, min_samples_leaf=1, min_samples_split=10, n_estimators=10


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


[CV 2/5; 1/250] END max_depth=20, max_features=10, min_samples_leaf=1, min_samples_split=10, n_estimators=10;, score=0.335 total time=   0.5s
[CV 3/5; 1/250] START max_depth=20, max_features=10, min_samples_leaf=1, min_samples_split=10, n_estimators=10


...

[CV 3/5; 250/250] END max_depth=15, max_features=50, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.297 total time=   7.5s
[CV 4/5; 250/250] START max_depth=15, max_features=50, min_samples_leaf=1, min_samples_split=2, n_estimators=50


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


[CV 4/5; 250/250] END max_depth=15, max_features=50, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.400 total time=   7.5s
[CV 5/5; 250/250] START max_depth=15, max_features=50, min_samples_leaf=1, min_samples_split=2, n_estimators=50


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


[CV 5/5; 250/250] END max_depth=15, max_features=50, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.572 total time=   7.6s


In [ ]:
resultados.best_params_

{'n_estimators': 50,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 25,
 'max_depth': 20}

In [ ]:
resultados.best_estimator_

RandomForestClassifier(max_depth=20, max_features=25, n_estimators=50,
                       random_state=26)

In [ ]:
rf_model = RandomForestClassifier(max_depth=20, max_features=25, n_estimators=50,
                       random_state=26)
modelo_rf = rf_model.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
print(f'El mejor score de validación obtenido es {top_k_accuracy_score(y_valid, rf_model.predict_proba(X_valid), k=2, labels=modelo_rf.classes_)}')

El mejor score de validación obtenido es 0.4987445072190835


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
